In [109]:
import pandas as pd
import numpy as np

In [110]:
df = pd.read_csv("padres_pitch_data.csv")

C:\Users\jamla\AppData\Local\Temp\ipykernel_26540\2281706610.py:1: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("padres_pitch_data.csv")


In [111]:
pd.set_option('display.max_columns', None)

In [112]:
df.head(4)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,pitcher_name,<html>
0,SL,2025-05-17,88.0,2.63,5.84,"Morejon, Adrian",456781.0,670970.0,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Donovan Solano strikes out swinging.,R,R,L,SD,SEA,S,2.0,NaN,1.0,2.0,2025.0,-0.51,-0.02,-1.25,1.60,NaN,NaN,668227.0,0.0,7.0,Top,NaN,NaN,NaN,NaN,NaN,NaN,-8.376075,-127.891673,-4.093214,-3.716814,28.163569,-31.739523,3.17,1.47,NaN,NaN,NaN,88.1,2757.0,6.6,777875.0,553869.0,657757.0,630105.0,592518.0,593428.0,518792.0,701538.0,665487.0,53.92,NaN,0.0,0.0,1.0,0.0,0.0,NaN,50.0,6.0,Slider,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN,284.0,0.035,-0.184,NaN,NaN,NaN,0.184,NaN,-1.0,1.0,0.263,0.737,26.0,37.0,26.0,38.0,1.0,0.0,3.0,1.0,NaN,NaN,2.99,-0.51,0.51,NaN,NaN,NaN,NaN,NaN,NaN,Adrian Morejon,NaN
1,SI,2025-05-17,96.1,2.27,5.97,"Morejon, Adrian",456781.0,670970.0,NaN,foul,NaN,NaN,NaN,NaN,4.0,Donovan Solano strikes out swinging.,R,R,L,SD,SEA,S,NaN,NaN,1.0,2.0,2025.0,1.22,0.94,-0.75,2.54,NaN,NaN,668227.0,0.0,7.0,Top,NaN,NaN,NaN,NaN,NaN,NaN,-10.754880,-139.463051,-5.577701,18.366087,31.506576,-18.842459,3.17,1.47,NaN,NaN,NaN,NaN,NaN,NaN,777875.0,553869.0,657757.0,630105.0,592518.0,593428.0,518792.0,701538.0,665487.0,54.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,5.0,Sinker,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,-1.0,1.0,0.263,0.737,26.0,37.0,26.0,38.0,1.0,0.0,3.0,1.0,NaN,NaN,1.55,1.22,-1.22,NaN,NaN,NaN,NaN,NaN,NaN,Adrian Morejon,NaN
2,SL,2025-05-17,86.4,2.58,5.98,"Morejon, Adrian",456781.0,670970.0,NaN,ball,NaN,NaN,NaN,NaN,12.0,Donovan Solano strikes out swinging.,R,R,L,SD,SEA,B,NaN,NaN,0.0,2.0,2025.0,-0.50,0.10,1.10,3.39,NaN,NaN,668227.0,0.0,7.0,Top,NaN,NaN,NaN,NaN,NaN,NaN,-2.499245,-125.916363,-0.188762,-4.892019,24.755810,-31.245104,3.21,1.41,NaN,NaN,NaN,NaN,NaN,NaN,777875.0,553869.0,657757.0,630105.0,592518.0,593428.0,518792.0,701538.0,665487.0,54.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,4.0,Slider,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,Standard,Standard,NaN,0.000,0.033,NaN,NaN,NaN,-0.033,NaN,-1.0,1.0,0.263,0.737,26.0,37.0,26.0,38.0,1.0,0.0,3.0,1.0,NaN,NaN,2.93,-0.50,0.50,NaN,NaN,NaN,NaN,NaN,NaN,Adrian Morejon,NaN
3,SI,2025-05-17,97.3,2.22,5.98,"Morejon, Adrian",456781.0,670970.0,NaN,foul,NaN,NaN,NaN,NaN,3.0,Donovan Solano strikes out swinging.,R,R,L,SD,SEA,S,NaN,NaN,0.0,2.0,2025.0,1.25,1.16,0.52,3.20,NaN,NaN,668227.0,0.0,7.0,Top,NaN,NaN,NaN,NaN,NaN,NaN,-7.406580,-141.334185,-4.557035,18.460529,35.057755,-15.734633,3.17,1.47,NaN,NaN,NaN,97.0,2395.0,6.

In [113]:
df = df[['pitch_type', 'game_date', 'pitcher', 'batter', 'player_name', 'game_pk', 'pitch_number', 'at_bat_number','inning', 'balls', 'strikes', 'zone', 'outs_when_up', 'inning_topbot', 'fld_score', 'bat_score', 'on_1b', 'on_2b', 'on_3b', 'if_fielding_alignment', 'of_fielding_alignment', 'plate_x', 'plate_z']]

In [114]:
df[['pitcher', 'batter', 'game_pk']] = df[['pitcher', 'batter', 'game_pk']].astype('Int64')

In [115]:
df_filt = df.dropna(subset=['pitch_type', 'zone'])

In [116]:
df_filt = df_filt.sort_values(by=['game_pk','pitcher', 'at_bat_number', 'pitch_number'],
                             ascending=True)

In [117]:
df_filt['pitch_count'] = df_filt.groupby(['game_pk','pitcher']).cumcount()+1

In [118]:
df_filt = df_filt.sort_values(by=['game_pk', 'at_bat_number', 'pitch_number'], ascending = True)

# Group by game, at-bat, and pitcher to account for pitcher changes mid-at-bat
df_filt['prev_pitch_type'] = (
    df_filt.groupby(['game_pk', 'at_bat_number', 'pitcher'])['pitch_type']
    .shift(1)
)

In [119]:
df_filt.head(20)

,pitch_type,game_date,pitcher,batter,player_name,game_pk,pitch_number,at_bat_number,inning,balls,strikes,zone,outs_when_up,inning_topbot,fld_score,bat_score,on_1b,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,pitch_count,prev_pitch_type
17506,ST,2022-07-27,592094,641820,"Adam, Jason",661053,1.0,59.0,7.0,0.0,0.0,14.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.51,0.80,1,NaN
17505,ST,2022-07-27,592094,641820,"Adam, Jason",661053,2.0,59.0,7.0,1.0,0.0,6.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.38,2.68,2,ST
17504,ST,2022-07-27,592094,641820,"Adam, Jason",661053,3.0,59.0,7.0,1.0,1.0,14.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,1.06,0.23,3,ST
17503,CH,2022-07-27,592094,641820,"Adam, Jason",661053,4.0,59.0,7.0,2.0,1.0,13.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,-1.10,1.43,4,ST
17502,ST,2022-07-27,592094,641820,"Adam, Jason",661053,5.0,59.0,7.0,2.0,2.0,9.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.64,1.97,5,CH
17501,CH,2022-07-27,592094,623993,"Adam, Jason",661053,1.0,63.0,8.0,0.0,0.0,4.0,0.0,Bot,4.0,3.0,NaN,NaN,NaN,Infield shift,Standard,-0.42,2.26,6,NaN
17500,CH,2022-07-27,592094,623993,"Adam, Jason",661053,2.0,63.0,8.0,0.0,1.0,14.0,0.0,Bot,4.0,3.0,NaN,NaN,NaN,Infield shift,Standard,0.34,1.02,7,CH
17499,FF,2022-07-27,592094,623993,"Adam, Jason",661053,3.0,63.0,8.0,0.0,2.0,11.0,0.0,Bot,4.0,3.0,NaN,NaN,NaN,Infield shift,Standard,-0.20,5.37,8,CH
17498,CH,2022-07-27,592094,623993,"Adam, Jason",661053,4.0,63.0,8.0,1.0,2.0,7.0,0.0,Bot,4.0,3.0,NaN,NaN,NaN,Infield shift,Standard,-0.59,1.81,9,FF
17497,ST,2022-07-27,592094,663624,"Adam, Jason",661053,1.0,64.0,8.0,0.0,0.0,14.0,1.0,Bot,4.0,3.0,NaN,NaN,NaN,Infield shift,Standard,0.85,2.41,10,NaN


In [120]:
df_filt['prev_pitch_type'] = df_filt['prev_pitch_type'].fillna('firstPitch')

In [121]:
df_filt

,pitch_type,game_date,pitcher,batter,player_name,game_pk,pitch_number,at_bat_number,inning,balls,strikes,zone,outs_when_up,inning_topbot,fld_score,bat_score,on_1b,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,pitch_count,prev_pitch_type
17506,ST,2022-07-27,592094,641820,"Adam, Jason",661053,1.0,59.0,7.0,0.0,0.0,14.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.51,0.80,1,firstPitch
17505,ST,2022-07-27,592094,641820,"Adam, Jason",661053,2.0,59.0,7.0,1.0,0.0,6.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.38,2.68,2,ST
17504,ST,2022-07-27,592094,641820,"Adam, Jason",661053,3.0,59.0,7.0,1.0,1.0,14.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,1.06,0.23,3,ST
17503,CH,2022-07-27,592094,641820,"Adam, Jason",661053,4.0,59.0,7.0,2.0,1.0,13.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,-1.10,1.43,4,ST
17502,ST,2022-07-27,592094,641820,"Adam, Jason",661053,5.0,59.0,7.0,2.0,2.0,9.0,2.0,Bot,4.0,3.0,NaN,NaN,622761.0,Standard,Standard,0.64,1.97,5,CH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38864,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,1.0,50.0,6.0,0.0,0.0,3.0,2.0,Top,9.0,2.0,NaN,NaN,NaN,Standard,Strategic,0.78,2.72,10,firstPitch
38863,ST,2025-02-22,673513,649966,"Matsui, Yuki",779026,2.0,50.0,6.0,0.0,1.0,12.0,2.0,Top,9.0,2.0,NaN,NaN,NaN,Standard,Strategic,0.68,4.01,11,FS
38862,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,3.0,50.0,6.0,1.0,1.0,14.0,2.0,Top,9.0,2.0,NaN,NaN,NaN,Standard,Strategic,0.07,1.36,12,ST
38861,FF,2025-02-22,673513,649966,"Matsui, Yuki",779026,4.0,50.0,6.0,1.0,2.0,11.0,2.0,Top,9.0,2.0,NaN,NaN,NaN,Standard,Strategic,-1.28,3.02,13,FS


In [122]:
df_filt[['on_1b', 'on_2b', 'on_3b']] = df_filt[['on_1b', 'on_2b', 'on_3b']].notna().astype(int)

In [123]:
df_filt

,pitch_type,game_date,pitcher,batter,player_name,game_pk,pitch_number,at_bat_number,inning,balls,strikes,zone,outs_when_up,inning_topbot,fld_score,bat_score,on_1b,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,pitch_count,prev_pitch_type
17506,ST,2022-07-27,592094,641820,"Adam, Jason",661053,1.0,59.0,7.0,0.0,0.0,14.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.51,0.80,1,firstPitch
17505,ST,2022-07-27,592094,641820,"Adam, Jason",661053,2.0,59.0,7.0,1.0,0.0,6.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.38,2.68,2,ST
17504,ST,2022-07-27,592094,641820,"Adam, Jason",661053,3.0,59.0,7.0,1.0,1.0,14.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,1.06,0.23,3,ST
17503,CH,2022-07-27,592094,641820,"Adam, Jason",661053,4.0,59.0,7.0,2.0,1.0,13.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,-1.10,1.43,4,ST
17502,ST,2022-07-27,592094,641820,"Adam, Jason",661053,5.0,59.0,7.0,2.0,2.0,9.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.64,1.97,5,CH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38864,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,1.0,50.0,6.0,0.0,0.0,3.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.78,2.72,10,firstPitch
38863,ST,2025-02-22,673513,649966,"Matsui, Yuki",779026,2.0,50.0,6.0,0.0,1.0,12.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.68,4.01,11,FS
38862,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,3.0,50.0,6.0,1.0,1.0,14.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.07,1.36,12,ST
38861,FF,2025-02-22,673513,649966,"Matsui, Yuki",779026,4.0,50.0,6.0,1.0,2.0,11.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,-1.28,3.02,13,FS


In [124]:
df_filt['if_fielding_alignment'].unique()

array(['Standard', 'Infield shift', 'Strategic', 'Infield shade', nan],
      dtype=object)

In [125]:
df_filt['score_diff'] = df_filt['fld_score'] - df_filt['bat_score']

In [126]:
df_filt

,pitch_type,game_date,pitcher,batter,player_name,game_pk,pitch_number,at_bat_number,inning,balls,strikes,zone,outs_when_up,inning_topbot,fld_score,bat_score,on_1b,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,pitch_count,prev_pitch_type,score_diff
17506,ST,2022-07-27,592094,641820,"Adam, Jason",661053,1.0,59.0,7.0,0.0,0.0,14.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.51,0.80,1,firstPitch,1.0
17505,ST,2022-07-27,592094,641820,"Adam, Jason",661053,2.0,59.0,7.0,1.0,0.0,6.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.38,2.68,2,ST,1.0
17504,ST,2022-07-27,592094,641820,"Adam, Jason",661053,3.0,59.0,7.0,1.0,1.0,14.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,1.06,0.23,3,ST,1.0
17503,CH,2022-07-27,592094,641820,"Adam, Jason",661053,4.0,59.0,7.0,2.0,1.0,13.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,-1.10,1.43,4,ST,1.0
17502,ST,2022-07-27,592094,641820,"Adam, Jason",661053,5.0,59.0,7.0,2.0,2.0,9.0,2.0,Bot,4.0,3.0,0,0,1,Standard,Standard,0.64,1.97,5,CH,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38864,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,1.0,50.0,6.0,0.0,0.0,3.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.78,2.72,10,firstPitch,7.0
38863,ST,2025-02-22,673513,649966,"Matsui, Yuki",779026,2.0,50.0,6.0,0.0,1.0,12.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.68,4.01,11,FS,7.0
38862,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,3.0,50.0,6.0,1.0,1.0,14.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,0.07,1.36,12,ST,7.0
38861,FF,2025-02-22,673513,649966,"Matsui, Yuki",779026,4.0,50.0,6.0,1.0,2.0,11.0,2.0,Top,9.0,2.0,0,0,0,Standard,Strategic,-1.28,3.02,13,FS,7.0


In [127]:
df_filt.to_csv('pitcher_data_cleaned.csv')

In [ ]:
preprocessor = ColumnTransformer([
    ('dummify', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), make_column_selector(dtype_include=object)),
    ("standardize", StandardScaler(), ['BMI', 'PhysicalHealthDays', 'MentalHealthDays'])
], remainder='drop')

XGB = XGBClassifier(n_jobs=-1, learning_rate=1, n_estimators=100)

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('boosting', XGB)
])

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('boosting', XGB)
])

param_grid = {
    'boosting__max_depth': [3, 5, 7, 9, 12, 15], 'boosting__n_estimators': [50,100,150], 'boosting__learning_rate': [.25,.5,.75,1]}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)

grid_search.fit(X, y)

print("Best F1 score:", grid_search.best_score_)
print("Best parameters:", grid_search.best_params_)